In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from datetime import date

pd.set_option('display.max_columns', None)

In [3]:
co_zips = pd.read_csv('../Demographics/CO_zip_codes_2016.csv')
co_zips = co_zips[['zip_code']]
co_zips['zip_code'] = co_zips['zip_code'].astype(str)
co_zips.rename(columns={'zip_code':'ZIP Code'},inplace=True)
co_zips.set_index('ZIP Code',inplace=True)

In [44]:
zcta_income = pd.read_excel('../Demographics/MHI_ZCTA_ACS5YR1418.xlsx')
zcta_income = zcta_income[['ZCTA', 'Median Household Income']]
zcta_income = zcta_income.replace(r'ZCTA5 ','',regex=True)
zcta_income.rename(columns={'ZCTA': 'ZIP Code'}, inplace=True)
zcta_income.set_index('ZIP Code',inplace=True)

zcta_households = pd.read_excel('../Demographics/Households_ZCTA_ACS5YR201418.xlsx')
zcta_households = zcta_households[['ZCTA', 'Total Households']]
zcta_households = zcta_households.replace(r'ZCTA5 ','',regex=True)
zcta_households.rename(columns={'ZCTA': 'ZIP Code'}, inplace=True)
zcta_households.set_index('ZIP Code',inplace=True)

income_hhs = zcta_income.merge(zcta_households, how = 'outer',left_index=True, right_index=True)

In [46]:
co_zctas = income_hhs[income_hhs.index.isin(co_zips.index)]
co_zctas

,Median Household Income,Total Households
ZIP Code,,
80002,59099.0,8239
80003,65957.0,14420
80004,75794.0,14964
80005,101313.0,11176
80007,140610.0,4282
...,...,...
81654,60833.0,434
81655,NaN,15
81656,NaN,98


In [47]:
colorado_evs = pd.read_excel('../Transportation/co_ev_registrations_public.xlsx')
colorado_evs_current = colorado_evs[colorado_evs['DMV Snapshot']=='CO DMV Direct (10/1/2020)']
colorado_evs_current['# Registrations'] = colorado_evs_current.groupby('ZIP Code')['VIN Prefix'].transform('count')
colorado_evs_current = colorado_evs_current[['ZIP Code', '# Registrations']]
colorado_evs_current.drop_duplicates(subset=['ZIP Code'],inplace=True)
colorado_evs_current.set_index('ZIP Code', inplace=True)
colorado_evs_current.index = colorado_evs_current.index.astype(str)

#The next two lines would filter by registrations that are valid previous to today's date, and expire after today's date.
#colorado_evs_active = colorado_evs[(colorado_evs['Registration Expiration Date'] > today_date)]
#colorado_evs_active = colorado_evs_active[(colorado_evs_active['Registration Valid Date'] < today_date)]

<ipython-input-47-cf56ee8816bd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  colorado_evs_current['# Registrations'] = colorado_evs_current.groupby('ZIP Code')['VIN Prefix'].transform('count')


In [48]:
combined = co_zctas.merge(colorado_evs_current, how = 'left', left_index=True, right_index=True)
combined['# Registrations'] = combined['# Registrations'].fillna(0)

In [49]:
combined['MHI Percentile'] = combined['Median Household Income'].rank(pct=True)

In [53]:
combined.loc[combined['MHI Percentile']==1, 'MHI Percentile']=.999

In [56]:
combined.to_csv('../Evolved/my outputs/co_EVreg_ZCTAs.csv')